In [ ]:
from __future__ import division, print_function

import collections
import csv
import datetime
import xml.etree.ElementTree as ET

import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
import pickle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def prepare_dataset(segments, ph, history):
    '''
    ph = 6, 30 minutes ahead
    ph = 12, 60 minutes ahead
    '''
    features_list = []
    labels_list = []
    raw_glu_list = []
    
    # Iterate over each segment
    for segment_name, segment_df in segments.items():
        # Ensure all columns are of numeric type
        # segment_df['carb_effect'] = pd.to_numeric(segment_df['carb_effect'], errors='coerce')
        # segment_df['steps'] = pd.to_numeric(segment_df['steps'], errors='coerce')
        # segment_df['steps'] = segment_df['steps'] 
        segment_df['bolus_effect'] = pd.to_numeric(segment_df['bolus_effect'], errors='coerce')

        # Fill NaNs that might have been introduced by conversion errors
        segment_df.fillna(0, inplace=True)

        # Maximum index for creating a complete feature set
        max_index = len(segment_df) - (history-1+ph+1)  # Subtracting 22 because we need to predict index + 21 and need index + history-1 to exist
        
        # Iterate through the data to create feature-label pairs
        for i in range(max_index + 1):
            # Extracting features from index i to i+history-1
            features = segment_df.loc[i:i+history-1, ['glucose_value',  'bolus_effect']] # .values.flatten() # 'carb_effect', 'bolus_effect', 'steps'
            # Extracting label for index i+21
            # Do the label transform
            label = segment_df.loc[i+history-1+ph, 'glucose_value'] - segment_df.loc[i+history-1, 'glucose_value']
            
            raw_glu_list.append(segment_df.loc[i+history-1+ph, 'glucose_value'])
            features_list.append(features)
            labels_list.append(label)
            
    print("len of features_list " + str(len(features_list)))
    print("len of labels_list " + str(len(labels_list)))
    # new_labels_list = label_delta_transform(labels_list)    
    # print("after label transform. the len of label list "+str(len(new_labels_list)))    
    return features_list, labels_list, raw_glu_list

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


class StackedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
        super(StackedLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # First LSTM layer
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers=1, batch_first=True).to(device)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_prob).to(device)
        
        # Second LSTM layer
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, num_layers=1, batch_first=True).to(device)
        
        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size, 512).to(device)
        self.fc2 = nn.Linear(512, 128).to(device)
        self.fc3 = nn.Linear(128, output_size).to(device)
        
        # Activation functions
        self.relu = nn.ReLU()
        
    
    def forward(self, x):
        batch_size = x.size(0)  # Get the batch size from the input tensor

        # Initialize hidden and cell state for the first LSTM layer
        h0 = torch.zeros(1, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(1, batch_size, self.hidden_size).to(x.device)
        
        # First LSTM layer
        out, (hn, cn) = self.lstm1(x, (h0, c0))
        
        # Dropout layer
        out = self.dropout(out)
        
        # Initialize hidden and cell state for the second LSTM layer
        h1 = torch.zeros(1, batch_size, self.hidden_size).to(x.device)
        c1 = torch.zeros(1, batch_size, self.hidden_size).to(x.device)
        
        # Second LSTM layer
        out, (hn, cn) = self.lstm2(out, (h1, c1))
        
        # Fully connected layers
        out = out[:, -1, :]  # Get the last time step output
        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        
        return out


In [ ]:
# Load processed data
filename = "./processed_data/BIG_training_data_onlyCGM.pkl"
# Load the dictionary from the file
with open(filename, 'rb') as f:
    loaded_df_dict = pickle.load(f)

# Verify the content


print(loaded_df_dict['1segment_1'])
print(loaded_df_dict['1segment_2'])
step_updated_segments = loaded_df_dict





In [ ]:
# Prepare for training
features_list, labels_list, raw_glu_list = prepare_dataset(step_updated_segments, 6, 6)

# Build training and validation loader
features_array = np.array(features_list)
labels_array = np.array(raw_glu_list) # Maybe need to replace this

X_train, X_val, y_train, y_val = train_test_split(features_array, labels_array, test_size=0.2, shuffle= False)

In [ ]:
# Data Preparation (assuming X_train, y_train, X_val, y_val are numpy arrays)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False) # The original batch size = 128, however, training on 128 cannot get the model fully trained, so change to 32.
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# Data Preparation (assuming X_train, y_train, X_val, y_val are numpy arrays)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False) # The original batch size = 128, however, training on 128 cannot get the model fully trained, so change to 32.
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
input_size = 2 # Number of input features
hidden_size = 128  # Hidden vector size
num_layers = 2  # Number of LSTM layers
output_size = 1  # Single output
dropout_prob = 0.2  # Dropout probability

model = StackedLSTM(input_size, hidden_size, num_layers, output_size, dropout_prob) # input_size, hidden_size, num_layers, output_size, dropout_prob
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)

In [ ]:
num_epochs =100
for epoch in range(num_epochs):
    model.train()
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}')


    model.eval()
    with torch.no_grad():
        total_loss = 0
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets.float())
            total_loss += loss.item()
        
        avg_loss = total_loss / len(val_loader)
        print(f'Test Loss: {avg_loss:.4f}')



In [ ]:
# # save the model 
torch.save(model.state_dict(), './processed_data/stacked_lstm_model_cgm_only_6_6.pth')

In [ ]:
model.eval()
predictions = []
actuals = []
with torch.no_grad():
    for inputs, targets in val_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions.append(outputs)
        actuals.append(targets)

predictions = torch.cat(predictions).cpu().numpy()
actuals = torch.cat(actuals).cpu().numpy()


rmse = root_mean_squared_error(actuals,predictions)
print(f'RMSE on validation set: {rmse}')

In [ ]:
plt.plot(predictions[:700], label = 'predictions')
plt.plot(actuals[:700], label = 'actuals')
plt.legend()

# Testing

In [ ]:
def test_model(model, test_step_updated_segments):

    # Prepare for training
    features_list_test, labels_list_test, raw_glu_list_test = prepare_dataset(test_step_updated_segments, 6, 6)
    
    # Build training and validation loader
    features_array_test = np.array(features_list_test)
    labels_array_test = np.array(raw_glu_list_test) # Maybe need to replace this

    X_test, y_test = features_array_test, labels_array_test

    # Data Preparation (assuming X_train, y_train, X_val, y_val are numpy arrays)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32)

    # Create DataLoader
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            predictions.append(outputs)
            actuals.append(targets)

    predictions = torch.cat(predictions).cpu().numpy()
    actuals = torch.cat(actuals).cpu().numpy()

    rmse = root_mean_squared_error(actuals,predictions)
    print(f'RMSE on validation set: {rmse}')
    
    return predictions, actuals, rmse

In [ ]:
# load model from .pth
model_dir = "./processed_data/stacked_lstm_model_cgm_only_6_6.pth"
input_size = 2 # Number of input features
hidden_size = 128  # Hidden vector size
num_layers = 2  # Number of LSTM layers
output_size = 1  # Single output
dropout_prob = 0.2  # Dropout probability

model = StackedLSTM(input_size, hidden_size, num_layers, output_size, dropout_prob) # input_size, hidden_size, num_layers, output_size, dropout_prob
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)

model.load_state_dict(torch.load(model_dir))

In [ ]:
import glob




preds = []
trues = []
errors = []
fname = []

for file in glob.glob("./processed_data/*test*ONLY*"):
    test_filename = file
    with open(test_filename, 'rb') as f:
        test_loaded_df_dict = pickle.load(f)

    # Verify the content
    test_step_updated_segments = test_loaded_df_dict
    pred, true, rmse = test_model(model, test_step_updated_segments)
    preds.append(pred)
    trues.append(true)
    errors.append(rmse)
    fname.append(file.split('/')[-1].split('_')[0])

In [ ]:
# make a 3x 2 plot and plot the preds and actuals of each one in each and have the title be the filename and the rmse

fig, axs = plt.subplots(3, 2, figsize=(15, 15))

for i in range(3):
    for j in range(2):
        idx = i*2 + j
        axs[i, j].plot(preds[idx], label = 'predictions')
        axs[i, j].plot(trues[idx], label = 'actuals')
        axs[i, j].legend()
        axs[i, j].set_title(f'{fname[idx]} RMSE: {errors[idx]:.4f}')
        
plt.tight_layout()

In [ ]:
# convert this all to a table 
# save the table as a csv
# convert the fname and the rmse to a dataframe
pd.DataFrame({'fname': fname, 'rmse': errors}).to_csv('./')

In [ ]:
# old results
data = [
    {'fname': 559, 'rmse': 28.529722},
    {'fname': 591, 'rmse': 27.535217},
    {'fname': 570, 'rmse': 25.205898},
    {'fname': 563, 'rmse': 23.448462},
    {'fname': 588, 'rmse': 22.671024},
    {'fname': 575, 'rmse': 30.21184}
]

# plot new data and old data
old_data = pd.DataFrame(data)
new_data = pd.DataFrame({'fname': fname, 'rmse': errors})

old_data.fname = old_data.fname.astype(str)
new_data.fname = new_data.fname.astype(str)

In [ ]:
n_dat = old_data.merge(new_data, on='fname', suffixes=('_all', '_only_cgm'))

In [ ]:
new_data

In [ ]:
# plot these are bar plots 
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
n_dat.plot.bar(x='fname', y=['rmse_all', 'rmse_only_cgm'], ax=ax)
plt.title('Comparison of RMSE between all features and only CGM')

In [ ]:
# convert this all to a table 
# save the table as a csv
# convert the fname and the rmse to a dataframe
pd.DataFrame({'fname': fname, 'rmse': errors})

In [ ]:
# Prepare for training
features_list_test, labels_list_test, raw_glu_list_test = prepare_dataset(test_step_updated_segments, 6, 6)

# Build training and validation loader
features_array_test = np.array(features_list_test)
labels_array_test = np.array(raw_glu_list_test) # Maybe need to replace this

X_test, y_test = features_array_test, labels_array_test

# Data Preparation (assuming X_train, y_train, X_val, y_val are numpy arrays)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
model.eval()
predictions = []
actuals = []
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions.append(outputs)
        actuals.append(targets)

predictions = torch.cat(predictions).cpu().numpy()

actuals = torch.cat(actuals).cpu().numpy()


rmse = root_mean_squared_error(actuals,predictions)
print(f'RMSE on validation set: {rmse}')

In [ ]:
plt.plot(predictions[:700])
plt.plot(actuals[:700])